 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8890/notebooks/22-unet-different-initial-filters.ipynb#Load-libraries" data-toc-modified-id="Load-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load libraries</a></span></li><li><span><a href="http://localhost:8890/notebooks/22-unet-different-initial-filters.ipynb#Define-loss-functions" data-toc-modified-id="Define-loss-functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Define loss functions</a></span></li><li><span><a href="http://localhost:8890/notebooks/22-unet-different-initial-filters.ipynb#Define-models" data-toc-modified-id="Define-models-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Define models</a></span></li><li><span><a href="http://localhost:8890/notebooks/22-unet-different-initial-filters.ipynb#Training" data-toc-modified-id="Training-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="http://localhost:8890/notebooks/22-unet-different-initial-filters.ipynb#vert-cut" data-toc-modified-id="vert-cut-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>vert cut</a></span></li></ul></div>

# Load libraries

In [1]:
import cv2
import numpy as np
import pandas as pd

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Activation, UpSampling2D, BatchNormalization
from keras.optimizers import RMSprop, SGD, Adam
from keras.losses import binary_crossentropy
import keras.backend as K

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


# Define loss functions

In [2]:
def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score


def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss


def bce_dice_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss

# Define models

In [3]:
def get_unet(input_shape, num_classes=1):
    inputs = Input(shape=input_shape)

    down0 = Conv2D(24, (3, 3), padding='same')(inputs)
    down0 = BatchNormalization()(down0)
    down0 = Activation('relu')(down0)
    down0 = Conv2D(24, (3, 3), padding='same')(down0)
    down0 = BatchNormalization()(down0)
    down0 = Activation('relu')(down0)
    down0_pool = MaxPooling2D((2, 2), strides=(2, 2))(down0)

    down1 = Conv2D(64, (3, 3), padding='same')(down0_pool)
    down1 = BatchNormalization()(down1)
    down1 = Activation('relu')(down1)
    down1 = Conv2D(64, (3, 3), padding='same')(down1)
    down1 = BatchNormalization()(down1)
    down1 = Activation('relu')(down1)
    down1_pool = MaxPooling2D((2, 2), strides=(2, 2))(down1)

    down2 = Conv2D(128, (3, 3), padding='same')(down1_pool)
    down2 = BatchNormalization()(down2)
    down2 = Activation('relu')(down2)
    down2 = Conv2D(128, (3, 3), padding='same')(down2)
    down2 = BatchNormalization()(down2)
    down2 = Activation('relu')(down2)
    down2_pool = MaxPooling2D((2, 2), strides=(2, 2))(down2)

    down3 = Conv2D(256, (3, 3), padding='same')(down2_pool)
    down3 = BatchNormalization()(down3)
    down3 = Activation('relu')(down3)
    down3 = Conv2D(256, (3, 3), padding='same')(down3)
    down3 = BatchNormalization()(down3)
    down3 = Activation('relu')(down3)
    down3_pool = MaxPooling2D((2, 2), strides=(2, 2))(down3)

    down4 = Conv2D(512, (3, 3), padding='same')(down3_pool)
    down4 = BatchNormalization()(down4)
    down4 = Activation('relu')(down4)
    down4 = Conv2D(512, (3, 3), padding='same')(down4)
    down4 = BatchNormalization()(down4)
    down4 = Activation('relu')(down4)
    down4_pool = MaxPooling2D((2, 2), strides=(2, 2))(down4)

    center = Conv2D(768, (3, 3), padding='same')(down4_pool)
    center = BatchNormalization()(center)
    center = Activation('relu')(center)
    center = Conv2D(768, (3, 3), padding='same')(center)
    center = BatchNormalization()(center)
    center = Activation('relu')(center)

    up4 = UpSampling2D((2, 2))(center)
    up4 = concatenate([down4, up4], axis=3)
    up4 = Conv2D(512, (3, 3), padding='same')(up4)
    up4 = BatchNormalization()(up4)
    up4 = Activation('relu')(up4)
    up4 = Conv2D(512, (3, 3), padding='same')(up4)
    up4 = BatchNormalization()(up4)
    up4 = Activation('relu')(up4)
    up4 = Conv2D(512, (3, 3), padding='same')(up4)
    up4 = BatchNormalization()(up4)
    up4 = Activation('relu')(up4)

    up3 = UpSampling2D((2, 2))(up4)
    up3 = concatenate([down3, up3], axis=3)
    up3 = Conv2D(256, (3, 3), padding='same')(up3)
    up3 = BatchNormalization()(up3)
    up3 = Activation('relu')(up3)
    up3 = Conv2D(256, (3, 3), padding='same')(up3)
    up3 = BatchNormalization()(up3)
    up3 = Activation('relu')(up3)
    up3 = Conv2D(256, (3, 3), padding='same')(up3)
    up3 = BatchNormalization()(up3)
    up3 = Activation('relu')(up3)

    up2 = UpSampling2D((2, 2))(up3)
    up2 = concatenate([down2, up2], axis=3)
    up2 = Conv2D(128, (3, 3), padding='same')(up2)
    up2 = BatchNormalization()(up2)
    up2 = Activation('relu')(up2)
    up2 = Conv2D(128, (3, 3), padding='same')(up2)
    up2 = BatchNormalization()(up2)
    up2 = Activation('relu')(up2)
    up2 = Conv2D(128, (3, 3), padding='same')(up2)
    up2 = BatchNormalization()(up2)
    up2 = Activation('relu')(up2)

    up1 = UpSampling2D((2, 2))(up2)
    up1 = concatenate([down1, up1], axis=3)
    up1 = Conv2D(64, (3, 3), padding='same')(up1)
    up1 = BatchNormalization()(up1)
    up1 = Activation('relu')(up1)
    up1 = Conv2D(64, (3, 3), padding='same')(up1)
    up1 = BatchNormalization()(up1)
    up1 = Activation('relu')(up1)
    up1 = Conv2D(64, (3, 3), padding='same')(up1)
    up1 = BatchNormalization()(up1)
    up1 = Activation('relu')(up1)

    up0 = UpSampling2D((2, 2))(up1)
    up0 = concatenate([down0, up0], axis=3)
    up0 = Conv2D(24, (3, 3), padding='same')(up0)
    up0 = BatchNormalization()(up0)
    up0 = Activation('relu')(up0)
    up0 = Conv2D(24, (3, 3), padding='same')(up0)
    up0 = BatchNormalization()(up0)
    up0 = Activation('relu')(up0)
    up0 = Conv2D(24, (3, 3), padding='same')(up0)
    up0 = BatchNormalization()(up0)
    up0 = Activation('relu')(up0)

    classify = Conv2D(num_classes, (1, 1), activation='sigmoid')(up0)

    model = Model(inputs=inputs, outputs=classify)

    return model

In [4]:
model = get_unet(input_shape=(512,512,3))
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 512, 512, 3)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 512, 512, 24)  672         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 512, 512, 24)  96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 512, 512, 24)  0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

# Training

In [4]:
input_size = 512
max_epochs = 50
orig_width = 1918
orig_height= 1280
threshold  = 0.5

In [5]:
df_train = pd.read_csv('data/train_masks.csv')

In [6]:
ids_train = df_train['img'].map(lambda s: s.split('.')[0])

In [7]:
ids_train_split, ids_valid_split = train_test_split(ids_train, test_size=0.2, random_state=42)

In [8]:
def randomShiftScaleRotate(image, mask,
                           shift_limit=(-0.0625, 0.0625),
                           scale_limit=(-0.1, 0.2),
                           rotate_limit=(-0, 0),
                           aspect_limit=(0, 0),
                           borderMode=cv2.BORDER_CONSTANT,
                           u=0.5):
    if np.random.random() < u:
        height, width, channel = image.shape

        angle = np.random.uniform(rotate_limit[0], rotate_limit[1])  # degree
        scale = np.random.uniform(1 + scale_limit[0], 1 + scale_limit[1])
        aspect = np.random.uniform(1 + aspect_limit[0], 1 + aspect_limit[1])
        sx = scale * aspect / (aspect ** 0.5)
        sy = scale / (aspect ** 0.5)
        dx = round(np.random.uniform(shift_limit[0], shift_limit[1]) * width)
        dy = round(np.random.uniform(shift_limit[0], shift_limit[1]) * height)

        cc = np.math.cos(angle / 180 * np.math.pi) * sx
        ss = np.math.sin(angle / 180 * np.math.pi) * sy
        rotate_matrix = np.array([[cc, -ss], [ss, cc]])

        box0 = np.array([[0, 0], [width, 0], [width, height], [0, height], ])
        box1 = box0 - np.array([width / 2, height / 2])
        box1 = np.dot(box1, rotate_matrix.T) + np.array([width / 2 + dx, height / 2 + dy])

        box0 = box0.astype(np.float32)
        box1 = box1.astype(np.float32)
        mat = cv2.getPerspectiveTransform(box0, box1)
        image = cv2.warpPerspective(image, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
                                    borderValue=(
                                        0, 0,
                                        0,))
        mask = cv2.warpPerspective(mask, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
                                   borderValue=(
                                       0, 0,
                                       0,))

    return image, mask


In [ ]:
all_imgs  = {}
all_masks = {}
for id in ids_train:
    img  = cv2.imread('data/train/{}.jpg'.format(id))
    img  = cv2.resize(img, (input_size, input_size))
    mask = cv2.imread('data/train_masks/{}_mask.png'.format(id), cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (input_size, input_size))
    all_imgs[id]  = img
    all_masks[id] = mask

In [11]:
import random

In [12]:
def train_generator(train_batch_size):
    while True:
        this_ids_train_split = random.sample(list(ids_train_split), len(ids_train_split))
        for start in range(0, len(ids_train_split), train_batch_size):
            x_batch = []
            y_batch = []
            end = min(start + train_batch_size, len(ids_train_split))
            ids_train_batch = this_ids_train_split[start:end]
            for id in ids_train_batch:
                img  = all_imgs[id]
                mask = all_masks[id]
                img, mask = randomShiftScaleRotate(img, mask)
                mask = np.expand_dims(mask, axis=2)
                x_batch.append(img)
                y_batch.append(mask)
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch


In [13]:
def valid_generator(val_batch_size):
    while True:
        for start in range(0, len(ids_valid_split), val_batch_size):
            x_batch = []
            y_batch = []
            end = min(start + val_batch_size, len(ids_valid_split))
            ids_valid_batch = ids_valid_split[start:end]
            for id in ids_valid_batch.values:
                img  = all_imgs[id]
                mask = all_masks[id]
                mask = np.expand_dims(mask, axis=2)
                x_batch.append(img)
                y_batch.append(mask)
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch



In [14]:
train_batch_size = 8
val_batch_size   = 16

In [15]:
model.compile(optimizer=SGD(lr=0.01, momentum=0.9, decay=0.0005),
              loss=bce_dice_loss,
              metrics=[dice_coeff])

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/unet_mod.hdf5',
                             save_best_only=True,
                             save_weights_only=True),
             TensorBoard(log_dir='logs')]

history = model.fit_generator(generator=train_generator(train_batch_size),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(train_batch_size)),
                    epochs=max_epochs,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

Epoch 1/50
2243s - loss: 0.0925 - dice_coeff: 0.9512 - val_loss: 0.0569 - val_dice_coeff: 0.9738
Epoch 2/50
2214s - loss: 0.0288 - dice_coeff: 0.9854 - val_loss: 0.2110 - val_dice_coeff: 0.9130
Epoch 3/50
2216s - loss: 0.0238 - dice_coeff: 0.9879 - val_loss: 0.0236 - val_dice_coeff: 0.9878
Epoch 4/50
2216s - loss: 0.0213 - dice_coeff: 0.9892 - val_loss: 0.0209 - val_dice_coeff: 0.9890
Epoch 5/50
2215s - loss: 0.0198 - dice_coeff: 0.9900 - val_loss: 0.0188 - val_dice_coeff: 0.9903
Epoch 6/50
2214s - loss: 0.0190 - dice_coeff: 0.9904 - val_loss: 0.0188 - val_dice_coeff: 0.9904
Epoch 7/50
2214s - loss: 0.0183 - dice_coeff: 0.9907 - val_loss: 0.0181 - val_dice_coeff: 0.9905
Epoch 8/50
2214s - loss: 0.0172 - dice_coeff: 0.9913 - val_loss: 0.0182 - val_dice_coeff: 0.9904
Epoch 10/50
2216s - loss: 0.0169 - dice_coeff: 0.9915 - val_loss: 0.0161 - val_dice_coeff: 0.9916
Epoch 11/50
2216s - loss: 0.0166 - dice_coeff: 0.9916 - val_loss: 0.0155 - val_dice_coeff: 0.9919
Epoch 12/50
2212s - loss: 0.

In [16]:
model.load_weights('./weights/unet_mod.hdf5')

In [17]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

[0.013427260192751533, 0.9930361006957843]

In [18]:
model.compile(optimizer=RMSprop(lr=0.0001),loss=bce_dice_loss,metrics=[dice_coeff])

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=5,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=3,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/unet_mod2.hdf5',
                             save_best_only=True,
                             save_weights_only=True)]

history = model.fit_generator(generator=train_generator(train_batch_size),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(train_batch_size)),
                    epochs=max_epochs,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

Epoch 1/50
2180s - loss: 0.0424 - dice_coeff: 0.9819 - val_loss: 0.0206 - val_dice_coeff: 0.9892
Epoch 2/50
2173s - loss: 0.0221 - dice_coeff: 0.9893 - val_loss: 0.0203 - val_dice_coeff: 0.9895
Epoch 3/50


In [17]:
model.load_weights('./weights/unet_mod2.hdf5')

In [20]:
val_batch_size   = 32

In [21]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

[0.0098109628258671883, 0.99481969462864994]

# vert cut

In [4]:
model = get_unet(input_shape=(1280,1024,3))

In [5]:
df_train = pd.read_csv('data/train_masks.csv')
ids_train = df_train['img'].map(lambda s: s.split('.')[0])
ids_train_split, ids_valid_split = train_test_split(ids_train, test_size=0.2, random_state=42)
ids_train_split = list(ids_train_split)
ids_valid_split = list(ids_valid_split)

In [6]:
import random

In [7]:
def train_generator(train_batch_size):
    while True:
        this_ids_train_split = random.sample(ids_train_split, len(ids_train_split))
        for start in range(0, len(ids_train_split), train_batch_size):
            x_batch = []
            y_batch = []
            end = min(start + train_batch_size, len(ids_train_split))
            ids_train_batch = this_ids_train_split[start:end]
            for id in ids_train_batch:
                img  = cv2.imread('data/train/{}.jpg'.format(id))
                mask = cv2.imread('data/train_masks/{}_mask.png'.format(id), cv2.IMREAD_GRAYSCALE)
                
                img1  = np.copy(img[:, 0:1024, :])
                mask1 = np.copy(mask[:, 0:1024])
                mask1 = np.expand_dims(mask1, axis=2)
                
                img2  = np.copy(img[:, 894:, :])
                img2 = cv2.flip(img2, 1)
                mask2 = np.copy(mask[:, 894:])
                mask2 = cv2.flip(mask2, 1)
                mask2 = np.expand_dims(mask2, axis=2)
                
                                
                x_batch.append(img1)
                y_batch.append(mask1)
                x_batch.append(img2)
                y_batch.append(mask2)
                
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

In [8]:
def valid_generator(val_batch_size):
    while True:
        for start in range(0, len(ids_valid_split), val_batch_size):
            
            x_batch = []
            y_batch = []
            
            end = min(start + val_batch_size, len(ids_valid_split))
            ids_valid_batch = ids_valid_split[start:end]
            for id in ids_valid_batch:
                img  = cv2.imread('data/train/{}.jpg'.format(id))
                mask = cv2.imread('data/train_masks/{}_mask.png'.format(id), cv2.IMREAD_GRAYSCALE)
                
                img1  = np.copy(img[:, 0:1024, :])
                mask1 = np.copy(mask[:, 0:1024])
                mask1 = np.expand_dims(mask1, axis=2)
                
                img2  = np.copy(img[:, 894:, :])
                img2  = cv2.flip(img2, 1)
                mask2 = np.copy(mask[:, 894:])
                mask2  = cv2.flip(mask2, 1)
                mask2 = np.expand_dims(mask2, axis=2)
                
                
                                
                x_batch.append(img1)
                y_batch.append(mask1)
                x_batch.append(img2)
                y_batch.append(mask2)
            
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

In [10]:
model.compile(optimizer=RMSprop(lr=0.0001), loss=bce_dice_loss, metrics=[dice_coeff])

In [9]:
val_batch_size   = 4

In [12]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

[0.32703295110016767, 0.91932135164620599]

In [12]:
max_epochs=10

In [10]:
train_batch_size = 1

In [16]:
model.compile(optimizer=SGDAccum(accum_iters=8, lr=0.00001, momentum=0.9, decay=1e-5, nesterov=True), loss=bce_dice_loss, metrics=[dice_coeff])

In [18]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/unet_mod_vert_cut.hdf5',
                             save_best_only=True,
                             save_weights_only=True)]

history = model.fit_generator(generator=train_generator(train_batch_size),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(train_batch_size)),
                    epochs=5,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

Epoch 1/5
6011s - loss: 0.2534 - dice_coeff: 0.9333 - val_loss: 0.1931 - val_dice_coeff: 0.9479
Epoch 2/5
5993s - loss: 0.1744 - dice_coeff: 0.9517 - val_loss: 0.1754 - val_dice_coeff: 0.9515
Epoch 3/5
6001s - loss: 0.1434 - dice_coeff: 0.9588 - val_loss: 0.1542 - val_dice_coeff: 0.9562
Epoch 4/5
6002s - loss: 0.1241 - dice_coeff: 0.9634 - val_loss: 0.1467 - val_dice_coeff: 0.9577
Epoch 5/5


ResourceExhaustedError: OOM when allocating tensor with shape[8,88,1280,1024]
	 [[Node: conv2d_25/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](concatenate_5/concat, conv2d_25/kernel/read)]]
	 [[Node: metrics_1/dice_coeff/Mean/_2645 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_2232_metrics_1/dice_coeff/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'conv2d_25/convolution', defined at:
  File "/home/sainath/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/sainath/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/sainath/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", 

In [19]:
model.load_weights('./weights/unet_mod_vert_cut.hdf5')

In [20]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/unet_mod_vert_cut2.hdf5',
                             save_best_only=True,
                             save_weights_only=True)]

history = model.fit_generator(generator=train_generator(train_batch_size),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(train_batch_size)),
                    epochs=5,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

Epoch 1/5
6005s - loss: 0.1114 - dice_coeff: 0.9663 - val_loss: 0.1503 - val_dice_coeff: 0.9563
Epoch 2/5
6005s - loss: 0.1025 - dice_coeff: 0.9684 - val_loss: 0.1345 - val_dice_coeff: 0.9598
Epoch 3/5
6005s - loss: 0.0956 - dice_coeff: 0.9700 - val_loss: 0.1307 - val_dice_coeff: 0.9600
Epoch 4/5


KeyboardInterrupt: 

In [21]:
model.compile(optimizer=SGDAccum(accum_iters=8, lr=0.0001, momentum=0.9, decay=1e-5, nesterov=True), loss=bce_dice_loss, metrics=[dice_coeff])

In [25]:
model.load_weights('./weights/unet_mod_vert_cut2.hdf5')

In [26]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/unet_mod_vert_cut3.hdf5',
                             save_best_only=True,
                             save_weights_only=True)]

history = model.fit_generator(generator=train_generator(train_batch_size),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(train_batch_size)),
                    epochs=5,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

Epoch 1/5
6007s - loss: 0.0737 - dice_coeff: 0.9751 - val_loss: 0.1118 - val_dice_coeff: 0.9642
Epoch 2/5
6008s - loss: 0.0537 - dice_coeff: 0.9801 - val_loss: 0.0848 - val_dice_coeff: 0.9696
Epoch 3/5
6008s - loss: 0.0452 - dice_coeff: 0.9824 - val_loss: 0.0903 - val_dice_coeff: 0.9677
Epoch 4/5
6009s - loss: 0.0402 - dice_coeff: 0.9839 - val_loss: 0.1182 - val_dice_coeff: 0.9550
Epoch 5/5


ResourceExhaustedError: OOM when allocating tensor with shape[8,88,1280,1024]
	 [[Node: conv2d_25/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](concatenate_5/concat, conv2d_25/kernel/read)]]
	 [[Node: loss_2/mul/_4555 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_2230_loss_2/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'conv2d_25/convolution', defined at:
  File "/home/sainath/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/sainath/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/sainath/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.l

In [27]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/unet_mod_vert_cut4.hdf5',
                             save_best_only=True,
                             save_weights_only=True)]

history = model.fit_generator(generator=train_generator(train_batch_size),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(train_batch_size)),
                    epochs=5,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

Epoch 1/5
6010s - loss: 0.0339 - dice_coeff: 0.9858 - val_loss: 0.0727 - val_dice_coeff: 0.9718
Epoch 2/5
6010s - loss: 0.0319 - dice_coeff: 0.9865 - val_loss: 0.0818 - val_dice_coeff: 0.9690
Epoch 3/5
6010s - loss: 0.0303 - dice_coeff: 0.9870 - val_loss: 0.0674 - val_dice_coeff: 0.9736
Epoch 4/5
6011s - loss: 0.0290 - dice_coeff: 0.9875 - val_loss: 0.0824 - val_dice_coeff: 0.9684
Epoch 5/5
6011s - loss: 0.0278 - dice_coeff: 0.9879 - val_loss: 0.0780 - val_dice_coeff: 0.9709


In [12]:
model.load_weights('./weights/unet_mod_vert_cut4.hdf5')

In [14]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

[0.067405287187348881, 0.9736140064734139]

In [18]:
from keras.optimizers import SGDAccum

In [19]:
model.compile(optimizer=SGDAccum(accum_iters=8, lr=0.001, momentum=0.9, decay=1e-5, nesterov=True), loss=bce_dice_loss, metrics=[dice_coeff])

In [20]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/unet_mod_vert_cut5.hdf5',
                             save_best_only=True,
                             save_weights_only=True)]

history = model.fit_generator(generator=train_generator(train_batch_size),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(train_batch_size)),
                    epochs=5,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

Epoch 1/5
6017s - loss: 0.0272 - dice_coeff: 0.9880 - val_loss: 0.0582 - val_dice_coeff: 0.9781
Epoch 2/5
6012s - loss: 0.0205 - dice_coeff: 0.9908 - val_loss: 0.1120 - val_dice_coeff: 0.9588
Epoch 3/5
6013s - loss: 0.0181 - dice_coeff: 0.9918 - val_loss: 0.0570 - val_dice_coeff: 0.9774
Epoch 4/5


KeyboardInterrupt: 

In [21]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/unet_mod_vert_cut6.hdf5',
                             save_best_only=True,
                             save_weights_only=True)]

history = model.fit_generator(generator=train_generator(train_batch_size),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(train_batch_size)),
                    epochs=5,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

Epoch 1/5
6013s - loss: 0.0164 - dice_coeff: 0.9925 - val_loss: 0.0639 - val_dice_coeff: 0.9758
Epoch 2/5
6013s - loss: 0.0153 - dice_coeff: 0.9929 - val_loss: 0.1151 - val_dice_coeff: 0.9611
Epoch 3/5
6013s - loss: 0.0147 - dice_coeff: 0.9932 - val_loss: 0.0748 - val_dice_coeff: 0.9731
Epoch 4/5
6013s - loss: 0.0141 - dice_coeff: 0.9935 - val_loss: 0.0762 - val_dice_coeff: 0.9743
Epoch 5/5


KeyboardInterrupt: 

In [22]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/unet_mod_vert_cut7.hdf5',
                             save_best_only=True,
                             save_weights_only=True)]

history = model.fit_generator(generator=train_generator(train_batch_size),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(train_batch_size)),
                    epochs=5,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

Epoch 1/5
6014s - loss: 0.0137 - dice_coeff: 0.9936 - val_loss: 0.0697 - val_dice_coeff: 0.9751
Epoch 2/5
6013s - loss: 0.0134 - dice_coeff: 0.9938 - val_loss: 0.2795 - val_dice_coeff: 0.9296
Epoch 3/5
6012s - loss: 0.0131 - dice_coeff: 0.9939 - val_loss: 0.0950 - val_dice_coeff: 0.9678
Epoch 4/5
6013s - loss: 0.0129 - dice_coeff: 0.9940 - val_loss: 0.0801 - val_dice_coeff: 0.9725
Epoch 5/5
6013s - loss: 0.0126 - dice_coeff: 0.9941 - val_loss: 0.1301 - val_dice_coeff: 0.9643
